In [ ]:
### Imports
%load_ext autoreload
%autoreload 2

# Append main folder
import sys
sys.path.append("../")

from glob import glob

from tqdm import tqdm
import pykep as pk
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.neighbors import NearestNeighbors

from vtk import vtkXMLUnstructuredGridReader
from vtk.util import numpy_support as VN

from mpl_toolkits import mplot3d
%matplotlib notebook

dt = 10 #timestep of the inspected simulations, affects time in plots
iterations = 3153600 # for time computation as well

# PK epochs of simulation time
starting_t = pk.epoch_from_string('2022-01-01 00:00:00.000')
end_t = pk.epoch(starting_t.mjd2000 + iterations * dt * pk.SEC2DAY)
total_days = end_t.mjd - starting_t.mjd

In [ ]:
# Load the original dataset
init_r = np.loadtxt("../data/pos.csv",delimiter=",")
init_v = np.loadtxt("../data/v.csv",delimiter=",")
init_planets = []
for r,v in zip(init_r,init_v):
    init_planets.append(pk.planet.keplerian(starting_t,r * 1000.0,v * 1000.0,pk.MU_EARTH,1.,1.,1.))

In [ ]:
# Load some simulation results
# files = glob("../results/1y_kep_thresholds_test/*.vtu")
files = glob("../results/1y_all_thresholds_test/*.vtu")

In [ ]:
rs,vs = [],[] #will hold r,v for whole simulation

# load vtks
for file in tqdm(files):
    reader = vtkXMLUnstructuredGridReader()
    reader.SetFileName(file)
    reader.Update()

    data = reader.GetOutput()

    v = VN.vtk_to_numpy(data.GetPointData().GetArray('velocity')).astype("double")
    r = VN.vtk_to_numpy(data.GetPoints().GetData()).astype("double")
    rs.append(r)
    vs.append(v)

In [ ]:
# Threshold for conjunction tracking as used in simulation
thresholds = [1,5,10,20,25,50,75,100,250,500]

In [ ]:
# Load the file tracking the conjunctions
# conj = pd.read_csv("../results/1y_kep_thresholds_test/conjunctionCounts.csv",names=thresholds)
conj = pd.read_csv("../results/1y_all_thresholds_test/conjunctionCounts.csv",names=thresholds)

In [ ]:
last_it = iterations #last it to look at
subsample = 100 # only sample every n-th iteration in plot

fig = plt.figure(figsize=(8,4),dpi=150)
fig.patch.set_facecolor('white')

# Time axis
t = np.linspace(0,end_t.mjd - starting_t.mjd,iterations)

# Iterate over thresholds and plot for each
for row in thresholds:
    plt.plot(t[:last_it][::subsample],conj[row].values[:last_it][::subsample] / 2,linewidth=3)
    
plt.legend([str(t) + "m" for t in thresholds],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
plt.title("Conjunction Thresholds Comparison")
plt.xlabel("Days")
plt.ylabel("# of Conjunctions")
plt.gca().set_yscale("log")

In [ ]:
# Compute min, max and mean of orbital elements over simulation
min_elements = [[],[],[],[],[],[]]
mean_elements = [[],[],[],[],[],[]]
max_elements = [[],[],[],[],[],[]]

for v_it,r_it in tqdm(zip(vs,rs),total=len(vs)):
    elements = [[],[],[],[],[],[]]
    for v,r in zip(v_it,r_it):
        a,e,i,W,w,E = pk.ic2par(r,v, pk.MU_EARTH)
        elements[0].append(abs(a))
        elements[1].append(abs(e))
        elements[2].append(abs(i))
        elements[3].append(abs(W))
        elements[4].append(abs(w))
        elements[5].append(abs(E))
    for i in range(6):
        min_elements[i].append(np.min(elements[i]))
        mean_elements[i].append(np.mean(elements[i]))
        max_elements[i].append(np.max(elements[i]))

In [ ]:
#t ime axis of the simulation
t = np.linspace(0,end_t.mjd - starting_t.mjd,len(mean_elements[0]))

# Plot for each orbital element
for idx,element in enumerate(["a","e","i","W","w","E"]):
    fig = plt.figure(figsize=(8,4),dpi=150)
    fig.patch.set_facecolor('white')
    plt.plot(t,mean_elements[idx],linewidth=1)
#     plt.plot(t,min_elements[idx],linewidth=1)
#     plt.plot(t,max_elements[idx],linewidth=1)
    plt.legend(["mean","min","max"],loc='upper center', bbox_to_anchor=(1.1,0.8), ncol=1, fancybox=True, shadow=True)
    plt.title("Evolution of "+element)
    plt.xlabel("Days")
    plt.ylabel(element)
#         plt.gca().set_yscale("log")

In [ ]:
# Compute a KNN to get distance to nearest neighbors over simulation

all_distances = []

for r_it in tqdm(rs,total=len(rs)):
    elements = [[],[],[],[],[],[]]
    knn = NearestNeighbors(n_neighbors=2).fit(r_it)
    distances,_ = knn.kneighbors(r_it)
    distances = distances[:,1] / 1000 # convert to km
    all_distances.append(distances)
    
all_distances = np.asarray(all_distances)

In [ ]:
# Subsample to look only at those below some threshold
small_distances = []
max_dist = 25
for dist in all_distances:
    small_distances.append(dist[dist < max_dist])

In [ ]:
# Compute distributions for each iteration
bins = 64
x = np.linspace(0, max_dist, bins)
y = np.linspace(0, total_days, len(all_distances))

X, Y = np.meshgrid(x, y)
Z = []
for dist in tqdm(small_distances):
    hist,bin_vals = np.histogram(dist, bins = x,density=False)
    hist = np.cumsum(hist)
    hist = np.concatenate([hist,[hist[-1]]]) # last value remains for bucket
    Z.append(hist)
x = np.concatenate([[0],x])
Z = np.asarray(Z)

In [ ]:
# Create beautiful plots
fig = plt.figure(figsize = (6,6),dpi=150)
ax = plt.axes(projection='3d')
ax.plot_surface(X, Y, Z,  rstride=1, cstride=1, cmap="plasma", edgecolor='none')
ax.view_init(elev=20., azim=120)
ax.set_xlabel('Closest Distance [km]')
ax.set_ylabel('Days')
ax.set_zlabel('Cumulative Frequency');
ax.set_xlim([0,max_dist])